<a href="https://colab.research.google.com/github/anandhc6/Assignment1/blob/main/sweep_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

In [73]:
import wandb
wandb.login()
#!wandb login --relogin
wandb.init(project='CS6910_Assignment1_Sweep',entity="anandh", name = 'class-samples-1')

wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


In [74]:
import numpy as np
from numpy import random
from keras.datasets import fashion_mnist
#import the required libraries
from matplotlib import pyplot
import math as math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


(X, y), (X_test, y_test) = fashion_mnist.load_data()

# Reshaping the data 
X = X.reshape(X.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)

# Normalizing
X = X/255.0
X_test = X_test/255.0

# Split the data X into a training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

#Layer dimensions
#la =[784,128,128,10]
#la=[784, 256, 256, 256, 10]
#la =[]
#L= len(la)-1  
#L=[]

#weights and bias empty initializing
np.random.seed(42)
params={}

In [75]:
def weight_initialize(weight_para,la):
    global params
    params ={}
    if weight_para == 'Xavier':
        for i in range(0,len(la)-1):
            params["W"+str(i)]=np.random.randn(la[i+1],la[i])*np.sqrt(2/la[i+1])
            params["b"+str(i)] = np.zeros((la[i+1], 1))
    if weight_para == 'random':
        for i in range(0,len(la)-1):
            params["W"+str(i)] = np.random.randn(la[i+1], la[i]) * 0.01
            params["b"+str(i)] = np.zeros((la[i+1], 1))

In [76]:
#Activation function     
def sigmoid(x):
     return 1. /(1. + np.exp(-x))

def activation_function(x,activation):
    if activation=='sigmoid':
        return (1.0 / (1.0 + np.exp(-x)))
    elif activation=='sigmoid_derivative':
        return (activation_function(x,'sigmoid') * (1 - activation_function(x,'sigmoid')))
    elif activation=='relu':
        return x * (x > 0)
    elif activation=='relu_derivative':
        x[x <= 0.0] = 0.0
        x[x > 0.0] = 1.0
        return x
    elif activation=='tanh':
        return np.tanh(x)
    elif activation=='tanh_derivative':
        return 1 - (activation_function(x,'tanh') ** 2)
    else:
        raise Exception("Invalid activation function",activation)
def softmax(x):
    soft = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
        numr = np.exp(x[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft


In [77]:
def feedforward(X, params,activation_para):

    L = len(params)//2  
    A = [None]*L # activations
    H = [None]*L # pre-activations
     
    k=0
    A[0]=np.dot(params["W"+str(k)],X)+params["b"+str(k)]
    H[0]=activation_function(A[k],activation_para)
   
    for k in range(1,L-1):
            A[k]=np.dot(params["W"+str(k)],H[k-1])+params["b"+str(k)]
            H[k]=activation_function(A[k],activation_para)
   
    k=L-1
    A[k]=np.dot(params["W"+str(k)],H[k-1])+params["b"+str(k)]
    H[k]=softmax(A[k])
   
    y=H[-1]
    return A,H,y



In [78]:

def back_propagation(X_train,y_train,params,A ,H,activation_para,loss):
    #Initialzing gradients
    L = len(params)//2
    #A = [None]*L # activations
    #H = [None]*L
    g_weights = [0]*L
    g_biases  = [0]*L
    g_a       = [0]*(L+1)
    g_h       = [0]*(L+1)
    n_samples = X_train.shape[0]  # Change depending on the dimensions of data
   
    # print(str(activation_para)+'_derivative')

    for k in range(L-1,-1,-1): #2,1,0
       
        if k == L-1:
            if loss == 'cross_entropy':
                g_a[k] = H[k]  - y_train  # keep or remove T depending on the dimensions of data
            elif loss == 'square_loss':
                g_a[k] = (H[k] - y_train) * H[k] * (1 - H[k])
           
       
        else:
            g_h[k] = (1/n_samples)*np.dot(params["W"+str(k+1)].T,g_a[k+1])
            if activation_para == 'sigmoid':
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],activation_function(A[k], str(activation_para)+'_derivative'))
            elif activation_para == 'tanh':
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],activation_function(A[k], str(activation_para)+'_derivative'))
            elif  activation_para == 'relu':
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],activation_function(A[k], str(activation_para)+'_derivative'))


        if k == 0:
            g_weights[k] = (1/n_samples)*np.dot(g_a[k],X_train.T)
        else:
            g_weights[k] = (1/n_samples)*np.dot(g_a[k],H[k-1].T)

        g_biases[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
    return g_weights,g_biases

In [79]:
def accuracy(x, y,params,activation_para):
    _,_,yhate=feedforward(x, params,activation_para)
    yh=np.argmax(yhate,axis=0)
    accuracy = accuracy_score(y, yh)*100
    print(accuracy)

In [80]:
def loss_accuracy(x,params,y,t,loss,activation_para, n_class=10):
    _,_,yhate=feedforward(x, params,activation_para)
    print(x.shape)
    print(y.shape)
    # ls=[]
    # accuracy=[]
    if loss == 'cross_entropy':
        one_hot = np.zeros((n_class, x.shape[1]))
        one_hot[y, np.array(list(range(x.shape[1])))] = 1
        ls= -np.sum(one_hot *np.log(yhate))/x.shape[1]
                  
    if loss == 'squared_loss':
        eIndicator = np.zeros((n_class, x.shape[1]))
        eIndicator[y, np.arange(x.shape[1])] = 1
        ls =np.sum((yhate - eIndicator)**2) / x.shape[1]
    
    yh=np.argmax(yhate,axis=0)
    accuracy = accuracy_score(y, yh)*100 
     
    # for p in range(len(ls)):
    #print("Accuracy of  = " + str(accuracy) +"loss of " +str(t) +" = " + str(ls) )
    #print("Accuracy of " +str(t) +" = " + str(accuracy) )
    return accuracy,ls

In [81]:
def vanillagd(x_train,y_train,x_val,y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L):
    t=0
    A=[]
    H=[]
    train_acc=[]
    train_loss=[]
    val_acc=[]
    val_loss=[]
    while t<epochs:
        for i in range(0,minibatch,1):
                    X_mini=x_train[(i*batch):((i+1)*batch)]
                    y_mini=y_train[(i*batch):((i+1)*batch)]
                    batch1 = X_mini.shape[0]
       
                    y_mini_one_hot = np.zeros((10, batch1))
                    y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
                    A,H,y_hat= feedforward(X_mini.T,params,activation_para)
                    g_weights,g_biases = back_propagation(X_mini.T,y_mini_one_hot,params,A ,H,activation_para,loss)
                    for j in range(0,L):
                      g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                      params['W'+str(j)]-=eta*g_weights[j]
                      params['b'+str(j)]-=eta*g_biases[j]
                   
        t+=1
        train_a,train_l = loss_accuracy(x_train.T, params, y_train, t, loss,activation_para)
        train_acc.append(train_a)
        train_loss.append(train_l)
        val_a,val_l = loss_accuracy(x_val.T, params, y_val, t, loss,activation_para)  
        val_acc.append(val_a)
        val_loss.append(val_l)
     
    return train_acc,train_loss,val_acc,val_loss
       

In [82]:
# MOMENTUM_GD optimizer 
        
def momentumGD(x_train,y_train,x_val,y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L):
    preW = {}
    preb = {}
    gamma = 0.9
    train_acc=[]
    train_loss=[]
    val_acc=[]
    val_loss=[]
    for k in range(0,L):
        preW['W'+str(k)]=np.zeros((la[k+1],la[k]))
        preb['b'+str(k)]=np.zeros((la[k+1],1))
    t=0
    while t<epochs:
        for i in range(0,minibatch,1):
            X_mini=x_train[(i*batch):((i+1)*batch)]
            y_mini=y_train[(i*batch):((i+1)*batch)]
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params,activation_para)
            g_weights,g_biases = back_propagation(X_mini.T,y_mini_one_hot,params,A ,H,activation_para,loss)
            for j in range(0,L):
                #g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                w=(gamma*preW['W'+str(j)])+eta*g_weights[j]
                b=(gamma*preb['b'+str(j)])+eta*g_biases[j]
                params['W'+str(j)]-=w-np.multiply(eta*weight_decay,params['W'+str(j)])
                params['b'+str(j)]-=b-np.multiply(eta*weight_decay,params['b'+str(j)])
                preW['W'+str(j)]=w
                preb['b'+str(j)]=b
        t+=1
        
        #Loss & accuracy
        train_a,train_l = loss_accuracy(x_train.T, params, y_train, t, loss,activation_para)
        train_acc.append(train_a)
        train_loss.append(train_l)
        val_a,val_l = loss_accuracy(x_val.T, params, y_val, t, loss,activation_para)  
        val_acc.append(val_a)
        val_loss.append(val_l)
    return train_acc,train_loss,val_acc,val_loss


In [83]:
#NAG optimizer  
       
def nag(x_train,y_train,x_val,y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L):
    t=0
    # batch=600
    gamma = 0.9
    preW={}
    preb={}
    train_acc=[]
    train_loss=[]
    val_acc=[]
    val_loss=[]
    for k in range(0,L):
        preW['W'+str(k)]=np.zeros((la[k+1],la[k]))
        preb['b'+str(k)]=np.zeros((la[k+1],1))
        # print(preW['W'+str(k)].shape)
    while t<epochs:
        for i in range(0,minibatch,1):
            X_mini=x_train[(i*batch):((i+1)*batch)]
            y_mini=y_train[(i*batch):((i+1)*batch)]
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params,activation_para)
            g_weights,g_biases = back_propagation(X_mini.T,y_mini_one_hot,params,A ,H,activation_para,loss)
            l_params={}
            for j in range(0,L):
                l_params['W'+str(j)]=params['W'+str(j)]-(gamma*preW['W'+str(j)])
                l_params['b'+str(j)]=params['b'+str(j)]-(gamma*preb['b'+str(j)])
            for j in range(0,L):
                g_weights,g_biases = back_propagation(X_mini.T,y_mini_one_hot,l_params,A ,H,activation_para,loss)
                #g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                w=(gamma*preW['W'+str(j)])+eta*g_weights[j]
                b=(gamma*preb['b'+str(j)])+eta*g_biases[j]
                params['W'+str(j)]-=w-np.multiply(eta*weight_decay,params['W'+str(j)])
                params['b'+str(j)]-=b-np.multiply(eta*weight_decay,params['b'+str(j)])
                preW['W'+str(j)]=w
                preb['b'+str(j)]=b
        t+=1
       
        #Loss & accuracy            
        train_a,train_l = loss_accuracy(x_train.T, params, y_train, t, loss,activation_para)
        train_acc.append(train_a)
        train_loss.append(train_l)
        val_a,val_l = loss_accuracy(x_val.T, params, y_val, t, loss,activation_para)  
        val_acc.append(val_a)
        val_loss.append(val_l)
     
    return train_acc,train_loss,val_acc,val_loss
       

In [84]:
#RMSprop optimizer
        
def rms(x_train,y_train,x_val,y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L):
    t=0
    gamma = 0.9
    eps = 1e-8
    beta = 0.9
    preW={}
    preb={}
    train_acc=[]
    train_loss=[]
    val_acc=[]
    val_loss=[]
    for k in range(0,L):
        preW['W'+str(k)]=np.zeros((la[k+1],la[k]))
        preb['b'+str(k)]=np.zeros((la[k+1],1))
    while t<epochs:
        for i in range(0,minibatch,1):
            X_mini=X_train[(i*batch):((i+1)*batch)]
            y_mini=y_train[(i*batch):((i+1)*batch)]
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params,activation_para)
            g_weights,g_biases=back_propagation(X_mini.T,y_mini_one_hot,params,A ,H,activation_para,loss)  
            for j in range(0,L):
                #g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                w=(beta*preW['W'+str(j)])+(1.0-beta)*(g_weights[j])**2
                b=(beta*preb['b'+str(j)])+(1.0-beta)*(g_biases[j])**2
                params['W'+str(j)]-=((eta/(np.sqrt(w+eps)))*g_weights[j])-np.multiply(eta*weight_decay,params['W'+str(j)])
                params['b'+str(j)]-=((eta/(np.sqrt(b+eps)))*g_biases[j])-np.multiply(eta*weight_decay,params['b'+str(j)])
                preW['W'+str(j)]=w
                preb['b'+str(j)]=b
        t+=1
        
        #Loss & accuracy
        train_a,train_l = loss_accuracy(x_train.T, params, y_train, t, loss,activation_para)
        train_acc.append(train_a)
        train_loss.append(train_l)
        val_a,val_l = loss_accuracy(x_val.T, params, y_val, t, loss,activation_para)  
        val_acc.append(val_a)
        val_loss.append(val_l)
    
    return train_acc,train_loss,val_acc,val_loss


In [85]:
#STOCHASTIC optimizer
       
def sgd(x_train,y_train,x_val,y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L):
    t=0
    train_acc=[]
    train_loss=[]
    val_acc=[]
    val_loss=[]
    while t<epochs:
        indexes=np.random.permutation(len(x_train))
        X_ran=x_train[indexes]
        Y_ran=y_train[indexes]
        for x,y in zip(X_ran,Y_ran):
            X_mini=np.array([x])
            y_mini=np.array([y])
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params,activation_para)
            g_weights,g_biases=back_propagation(X_mini.T,y_mini_one_hot,params,A ,H,activation_para,loss)
            for j in range(0,L):
                #g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                params['W'+str(j)]-=eta*g_weights[j]#-np.multiply(eta*weight_decay,params['W'+str(j)])
                params['b'+str(j)]-=eta*g_biases[j]#-np.multiply(eta*weight_decay,params['b'+str(j)])
        t+=1
       
        #Loss & accuracy
        train_a,train_l = loss_accuracy(x_train.T, params, y_train, t, loss,activation_para)
        train_acc.append(train_a)
        train_loss.append(train_l)
        val_a,val_l = loss_accuracy(x_val.T, params, y_val, t, loss,activation_para)  
        val_acc.append(val_a)
        val_loss.append(val_l) 
    return train_acc,train_loss,val_acc,val_loss
     

In [86]:
#ADAM optimizer
        
def adam(x_train,y_train,x_val,y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L):
    eps = 1e-8
    beta1 = 0.9
    beta2 = 0.999
    t=0
    preW={}
    preb={}
    mt={}
    vt={}
    it=0
    train_acc=[]
    train_loss=[]
    val_acc=[]
    val_loss=[]
    for k in range(0,L):
        preW['W'+str(k)]=np.zeros((la[k+1],la[k]))
        preb['b'+str(k)]=np.zeros((la[k+1],1))
    for k in range(0,L):
        mt['W'+str(k)]=np.zeros((la[k+1],la[k]))
        vt['b'+str(k)]=np.zeros((la[k+1],1))
    while t<epochs:
        for i in range(0,minibatch,1):
            X_mini=x_train[(i*batch):((i+1)*batch)]
            y_mini=y_train[(i*batch):((i+1)*batch)]
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params,activation_para)
            g_weights,g_biases=back_propagation(X_mini.T,y_mini_one_hot,params,A ,H,activation_para,loss)
            it+=1
            for j in range(0,L):
                #g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                mt['W'+str(j)]=beta1*mt['W'+str(j)]+(1-beta1)*g_weights[j]
                vt['b'+str(j)]=beta1*vt['b'+str(j)]+(1-beta1)*g_biases[j]
                preW['W'+str(j)]=beta2*preW['W'+str(j)]+(1-beta2)*(g_weights[j]**2)
                preb['b'+str(j)]=beta2*preb['b'+str(j)]+(1-beta2)*(g_biases[j]**2)
                mhatw=mt['W'+str(j)]/(1-(beta1)**it)
                mhatb=vt['b'+str(j)]/(1-(beta1)**it)
                vhatw=preW['W'+str(j)]/(1-(beta2)**it)
                vhatb=preb['b'+str(j)]/(1-(beta2)**it)
                params['W'+str(j)]-=((eta/(np.sqrt(vhatw+eps)))*mhatw)-np.multiply(eta*weight_decay,params['W'+str(j)])
                params['b'+str(j)]-=((eta/(np.sqrt(vhatb+eps)))*mhatb)-np.multiply(eta*weight_decay,params['b'+str(j)])
        t+=1
        
        #Loss & accuracy
        train_a,train_l = loss_accuracy(x_train.T, params, y_train, t, loss,activation_para)
        train_acc.append(train_a)
        train_loss.append(train_l)
        val_a,val_l = loss_accuracy(x_val.T, params, y_val, t, loss,activation_para)  
        val_acc.append(val_a)
        val_loss.append(val_l)
     
    return train_acc,train_loss,val_acc,val_loss


In [87]:
#NADAM optimizer
       
def nadam(x_train,y_train,x_val,y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L):

    print('inside optimizer nadam')
    print(la)
    eps = 1e-8
    beta1 = 0.9
    beta2 = 0.999
    t=0
    minibatch=math.ceil(x_train.shape[0]/batch)
    preW={}
    preb={}
    mt={}
    vt={}
    it=0
    train_acc=[]
    train_loss=[]
    val_acc=[]
    val_loss=[]
    for k in range(0,L):
        preW['W'+str(k)]=np.zeros((la[k+1],la[k]))
        preb['b'+str(k)]=np.zeros((la[k+1],1))
    for k in range(0,L):
        mt['W'+str(k)]=np.zeros((la[k+1],la[k]))
        vt['b'+str(k)]=np.zeros((la[k+1],1))
    while t<epochs:
        for i in range(0,minibatch,1):
            X_mini=x_train[(i*batch):((i+1)*batch)]
            y_mini=y_train[(i*batch):((i+1)*batch)]
            batch1 = X_mini.shape[0]
            y_mini_one_hot = np.zeros((10, batch1))
            y_mini_one_hot[y_mini, np.array(list(range(batch1)))] = 1
            A,H,y_hat=feedforward(X_mini.T,params,activation_para)
            g_weights,g_biases=back_propagation(X_mini.T,y_mini_one_hot,params,A ,H,activation_para,loss)
            it+=1
            for j in range(0,L):
                #g_weights[j]=g_weights[j]+weight_decay*params['W'+str(j)]
                mt['W'+str(j)]=beta1*mt['W'+str(j)]+(1-beta1)*g_weights[j]
                vt['b'+str(j)]=beta1*vt['b'+str(j)]+(1-beta1)*g_biases[j]
                preW['W'+str(j)]=beta2*preW['W'+str(j)]+(1-beta2)*(g_weights[j]**2)
                preb['b'+str(j)]=beta2*preb['b'+str(j)]+(1-beta2)*(g_biases[j]**2)
                mhatw=mt['W'+str(j)]/(1-(beta1)**it)
                mhatb=vt['b'+str(j)]/(1-(beta1)**it)
                vhatw=preW['W'+str(j)]/(1-(beta2)**it)
                vhatb=preb['b'+str(j)]/(1-(beta2)**it)
                mbarw=beta1*mhatw+(1.0-beta1)*g_weights[j]
                mbarb=beta1*mhatb+(1.0-beta1)*g_biases[j]
                params['W'+str(j)]-=((eta/(np.sqrt(vhatw+eps)))*mbarw)-np.multiply(eta*weight_decay,params['W'+str(j)])
                params['b'+str(j)]-=((eta/(np.sqrt(vhatb+eps)))*mbarb)-np.multiply(eta*weight_decay,params['b'+str(j)])
        t+=1
       
        #Loss & accuracy
    
        train_a,train_l = loss_accuracy(x_train.T, params, y_train, t, loss,activation_para)
        train_acc.append(train_a)
        train_loss.append(train_l)
        val_a,val_l = loss_accuracy(x_val.T, params, y_val, t, loss,activation_para)  
        val_acc.append(val_a)
        val_loss.append(val_l)
    return train_acc,train_loss,val_acc,val_loss
       

In [88]:
#FIT models        
def fit(X,y,batch,epochs,eta,weight_para,optimiser,loss,weight_decay,activation_para,la,L):      
    weight_para = 'Xavier'
    weight_initialize(weight_para,la)
    
    X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    
    #batch=100
    
    minibatch=math.ceil(X_train.shape[0]/batch)
    
    if optimiser=='momentumGD':
        print('calling optimizer momentum ')
        train_acc,train_loss,val_acc,val_loss=momentumGD(X_train,y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L)
    elif optimiser=='nadam':
        print('calling optimizer nadam ')
        train_acc,train_loss,val_acc,val_loss=nadam(X_train,Y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L)
    elif optimiser=='rms':
        print('calling optimizer rms ')
        train_acc,train_loss,val_acc,val_loss=rms(X_train,Y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L)
    elif optimiser=='sgd':
        print('calling optimizer sgd ')
        train_acc,train_loss,val_acc,val_loss=sgd(X_train,Y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L)
    elif optimiser=='adam':
        print('calling optimizer adam ')
        train_acc,train_loss,val_acc,val_loss=adam(X_train,Y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L)
    elif optimiser=='nag':
        print('calling optimizer nag ')
        train_acc,train_loss,val_acc,val_loss=nag(X_train,Y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L)
    elif optimiser=='vanillagd':
        train_acc,train_loss,val_acc,val_loss=vanillagd(X_train,Y_train,X_val,Y_val,eta,epochs,minibatch,batch,loss,weight_decay,activation_para,la,L)   
    
    return train_acc,train_loss,val_acc,val_loss

         
#train_acc,train_loss,val_acc,val_loss= fit(X,y,100,5,0.01,'Xavier','nadam','cross_entropy',0.0005,'tanh',la,L)

#print(train_acc,train_loss,val_acc,val_loss)

In [ ]:
features=784
n_class=10
loss='cross_entropy'
#la=[]
#L=0
#print("calling FF_fit")
def FF_fit():
    """
    This function is used for the hyperparameter tuning using wandb sweeps
    """
    # Default values for hyper-parameters
    config_defaults = {
        'epochs': 10,
        'batch': 64,
        'eta': 1e-3,
        'activation_para': 'relu',
        'optimiser': 'adam',
        'weight_para': 'Xavier',
        'weight_decay': 0,
        'no_of_neurons': 64,
        'no_of_hidden': 3
    }
    
    la=[]
    L=0
    #wandb.init()
    wandb.init(config=config_defaults)
    config = wandb.config
    
    no_of_neurons = config.no_of_neurons
    no_of_hidden = config.no_of_hidden
    weight_para = config.weight_para
    epochs = config.epochs
    batch = config.batch
    eta = config.eta
    activation_para = config.activation_para
    weight_decay = config.weight_decay
    optimiser = config.optimiser
    
    
    la =[features]+[no_of_neurons]*no_of_hidden+[n_class]
    print(la)
    L =len(la)-1
    
    name = "lr_"+str(config.eta)+"_hl_"+str(config.no_of_hidden)+"_bs_"+str(config.batch)+"_ac_"+str(config.activation_para)+"_ep_"+str(config.epochs)
    wandb.run.name = name
    # name = "lr_{}_af_{}_in_{}_op_{}_b_{}_wd_{}_ep_{}_nn_{}_nh_{}".format(eta, activation_para, weight_para, optimiser, batch, weight_decay, epochs, no_of_neurons, no_of_hidden)
    # print(name)
    # fit(X,y,100,5,0.01,'Xavier','nadam','cross_entropy',0.0005,'tanh')
    train_acc,train_loss,val_acc,val_loss=fit(X,y,batch,epochs,eta,weight_para,optimiser,loss,weight_decay,activation_para,la,L)

    #printing Accuracy and loss
    for p in range(len(train_acc)):
       print("Accuracy of  train= " + str(train_acc[p]) +"   Loss of train = " + str(train_loss[p]) + "   Accuracy of val= " + str(val_acc[p]) + "  Accuracy of val= " + str(val_loss[p]))

    test_acc=accuracy(X_test.T,y_test,params,activation_para)
    print("Accuracy of  test_data= " + str(test_acc))

    for i in range(len(train_acc)):
      wandb.log({'train_acc':train_acc[i], 'train_loss':train_loss[i], 'val_acc':val_acc[i] ,'val_loss':val_loss[i], 'epochs': i+1})
    wandb.log({'test_acc' : test_acc})
    wandb.run.save()
    wandb.run.finish()
    
sweep_config = {
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [10,20,40]
        },
        'no_of_hidden': {
            'values': [3,4,5]
        },
        'no_of_neurons': {
            'values': [128,256]
        },
        'weight_decay': {
            'values': [0,0.0005]
        },
        'eta': {
            'values': [ 0.01,0.1,0.3]
        },
        'optimiser': {
            'values': ['rms','momentumGD','nag','sgd','adam','nadam']
        },
        'batch':{
            'values': [16,32,64,100]
        },
        'weight_para':{
            'values': ['Xavier','random']
        },
        'activation_para': {
            'values': ['tanh','sigmoid','relu']
        }
    }
}
    
wandb.init(project='cs6910-assignment1', name = 'class-samples-1')
#wandb.init(entity="anandh", project = "CS6910_Assignment1_Sweep")
sweep_id=wandb.sweep(sweep_config,entity="anandh", project = "CS6910_Assignment1_Sweep")
#sweep_id= "w7mehhhy"
#%start wandb sweep
wandb.agent(sweep_id, FF_fit, count = 1)
#wandb.agent(sweep_id, FF_fit, entity="anandh", project = "CS6910_Assignment1_Sweep", count = 5)


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 4kdj2a9g
Sweep URL: https://wandb.ai/anandh/CS6910_Assignment1_Sweep/sweeps/4kdj2a9g


wandb: Agent Starting Run: qhueh6wu with config:
wandb: 	activation_para: sigmoid
wandb: 	batch: 16
wandb: 	epochs: 20
wandb: 	eta: 0.1
wandb: 	no_of_hidden: 5
wandb: 	no_of_neurons: 256
wandb: 	optimiser: nadam
wandb: 	weight_decay: 0
wandb: 	weight_para: random


[784, 256, 256, 256, 256, 256, 10]
calling optimizer nadam 
inside optimizer nadam
[784, 256, 256, 256, 256, 256, 10]
(784, 54000)
(54000,)
(784, 6000)
(6000,)
(784, 54000)
(54000,)
(784, 6000)
(6000,)
(784, 54000)
(54000,)
(784, 6000)
(6000,)
(784, 54000)
(54000,)
(784, 6000)
(6000,)
(784, 54000)
(54000,)
(784, 6000)
(6000,)
(784, 54000)
(54000,)
(784, 6000)
(6000,)
(784, 54000)
(54000,)
(784, 6000)
(6000,)
(784, 54000)
(54000,)
(784, 6000)
(6000,)


In [ ]:
(X_mnist_full, y_mnist_full), (X_test_mnist_full, y_test_mnist_full) = mnist.load_data()

# Reshaping the data 
X_mnist_full = X.reshape(X.shape[0], 784)
X_test_mnist_full = X_test.reshape(X_test.shape[0], 784)

# Normalizing
X_mnist_full = X_mnist_full/255.0
X_test_mnist_full = X_test_mnist_full/255.0

#layer_dimensions
la_mnist_full=[784, 256, 256, 256, 10]
L_mnist_full= len(la)-1

# Split the data X into a training set and validation set
X_train_mnist_full, X_val_mnist_full, y_train_mnist_full, y_val_mnist_full = train_test_split(X_mnist_full, y_mnist_full, test_size=0.1, random_state=42)

train_acc_mnist_full,train_loss_mnist_full,val_acc_mnist_full,val_loss_mnist_full= fit(X_mnist_full,y_mnist_full,100,5,0.01,'Xavier','nadam','cross_entropy',0.0005,'tanh',la_mnist_full,L_mnist_full)

#train_acc,train_loss,val_acc,val_loss=fit(X,y,batch,epochs,eta,weight_para,optimiser,loss,weight_decay,activation_para)

#printing Accuracy and loss
for p in range(train_acc_mnist_full):
  print("Accuracy of  train= " + str(train_acc_mnist_full[p]) +"Loss of train = " + str(train_loss_mnist_full[p]) + "Accuracy of val=" + str(val_acc_mnist_full[p]) + "Accuracy of val=" + str(val_loss_mnist_full[p]))

   